In [1]:
import pandas as pd
import os
import json 

In [2]:
df = pd.DataFrame(columns=['optimizer', 'learning_rate', 'iteration', 'test_accuracy'])

In [3]:
for root, dirs, files in os.walk('./logs'):
    for dir in dirs:
        if dir.startswith('wav2vec2-base-superb-ks-') and "opt" in dir:
            optimizer = dir.split('_')[5]
            learning_rate = dir.split('_')[7]
            iteration = dir.split('_')[-1]
            with open(f'{root}/{dir}/data.json') as f:
                data = json.load(f)
            test_accuracy = data['test_accuracy']
            df = pd.concat([df, pd.DataFrame({'optimizer': [optimizer], 'learning_rate': [learning_rate], 'iteration': [iteration], 'test_accuracy': [test_accuracy]})])
df = df.reset_index(drop=True)

In [4]:
df.head()

,optimizer,learning_rate,iteration,test_accuracy
0,adam,0.0001,0,0.943164
1,adam,0.0001,1,0.941628
2,adam,0.0001,2,0.937020
3,adam,0.0001,3,0.932412
4,adam,0.0001,4,0.930876


In [5]:
# group by optimizer and learning rate and calculate mean and std
df['learning_rate'] = df['learning_rate'].astype(float)
df['iteration'] = df['iteration'].astype(int)
df['test_accuracy'] = df['test_accuracy'].astype(float)
df_grouped = df.groupby(['optimizer', 'learning_rate']).agg({'test_accuracy': ['mean', 'std']}).reset_index()
df_grouped.columns = ['optimizer', 'learning_rate', 'test_accuracy_mean', 'test_accuracy_std']
df_grouped = df_grouped.sort_values(by='test_accuracy_mean', ascending=False)   

In [6]:
df_grouped

,optimizer,learning_rate,test_accuracy_mean,test_accuracy_std
0,adam,0.0001,0.937020,0.005431
5,sgd,0.0100,0.929032,0.002748
4,sgd,0.0010,0.921352,0.008301
1,adam,0.0010,0.907220,0.007802
3,sgd,0.0001,0.574808,0.096300
2,adam,0.0100,0.235637,0.350876


In [7]:
df_grouped['test_accuracy_mean'] = df_grouped['test_accuracy_mean'].round(3)
df_grouped['test_accuracy_std'] = df_grouped['test_accuracy_std'].round(3)

In [8]:
df_grouped

,optimizer,learning_rate,test_accuracy_mean,test_accuracy_std
0,adam,0.0001,0.937,0.005
5,sgd,0.0100,0.929,0.003
4,sgd,0.0010,0.921,0.008
1,adam,0.0010,0.907,0.008
3,sgd,0.0001,0.575,0.096
2,adam,0.0100,0.236,0.351
